In [137]:
#Import required libraries
import pandas as pd
import numpy as np
import os
import re

from sklearn.model_selection import train_test_split

from datetime import datetime

In [139]:
#Load Dataset

customers_df=pd.read_csv('customers.csv')
transactions_df=pd.read_csv('transactions_train.csv')
articles_df=pd.read_csv('articles.csv')

In [9]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [10]:
transactions_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [13]:
articles_df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [15]:
print("Shape of transactions_df ",transactions_df.shape)
print("Shape of customers_df ",customers_df.shape)
print("Shape of articles_df ",articles_df.shape)

Shape of transactions_df  (31788324, 5)
Shape of customers_df  (1371980, 7)
Shape of articles_df  (105542, 25)


## Creating a Subset of the Data

In [118]:
# Taking a Subset of the customers
# Later we extract the transactions info relevant to these customers
# and work with that
customer_subset_df = customers_df.sample(frac=0.025)

In [119]:
customer_subset_df.shape

(34300, 7)

In [120]:
# Get customer id included in the subset
customer_ids=customer_subset_df.customer_id
customer_ids= list(customer_ids)

In [121]:
# Drop Duplicate rows
transactions_df = transactions_df.drop_duplicates()

## Extract relevant transactions
re=transactions_df['customer_id'].isin(customer_ids)
transactions_subset_df = transactions_df[re]
transactions_subset_df.shape

(715465, 5)

In [122]:
# Extract relevant article information
article_ids=list(transactions_subset_df.article_id.unique())
re=article_df['article_id'].isin(article_ids)
article_subset_df = article_df[re]
article_subset_df.shape

(66558, 25)

## Splitting the Data into Train and Test

In [123]:
# Convert the t_dat column of the transaction to datetime format
transactions_subset_df["t_dat"]=pd.to_datetime(transactions_subset_df['t_dat'], infer_datetime_format=True)

print("Earliest Date :",transactions_subset_df['t_dat'].min())
print("Latest Date : ",transactions_subset_df['t_dat'].max())

Earliest Date : 2018-09-20 00:00:00
Latest Date :  2020-09-22 00:00:00


C:\Users\chnan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [124]:
# Split the transactions data into train and test data by taking entire duration except last week as training data and the last week as test data
transactions_train = transactions_subset_df[(transactions_subset_df['t_dat'] <= '2020-09-15')]

transactions_test = transactions_subset_df[(transactions_subset_df['t_dat'] > '2020-09-15')]
transactions_test = transactions_test[transactions_test['customer_id'].isin(transactions_train["customer_id"])]

In [125]:
# Transforming test data to only contain customer_id and purchased article_ids
test=transactions_test.groupby("customer_id")
groups_test = dict(list(test))

rows = []
for i in groups_test.keys():
    rows.append({"customer_id":i,"products":groups_test[i]["article_id"].values})

test_data=pd.DataFrame(rows,columns=["customer_id","products"])

In [126]:
test_data.head()

,customer_id,products
0,006908fb1f581e64408d2c5a57bf65e06b26939dce50b8...,"[907149001, 714790024, 640021012, 918890001]"
1,00c557555ebb101c31b19dc53d4c8ad1a84933655b273c...,[914118001]
2,00f7b7feda6810b83c34073f2651b566846ad6c30f7649...,"[732842014, 732842002, 790365001, 921380001]"
3,012910345252a27e8a8be71e41a15e25d73645413a6cec...,"[669091031, 669091031]"
4,012a71bf4f614c1163c523885eba6f2f7992d8be928ec3...,"[873771002, 673677022]"


## PreProcessing of Data

In [127]:
# Checking for Missing Values in Customers data

customer_subset_df.isna().sum()

customer_id                   0
FN                        22289
Active                    22610
club_member_status          145
fashion_news_frequency      410
age                         402
postal_code                   0
dtype: int64

In [128]:
## In the Age column, we impute the NaN values with the average age of the customer

customers = customer_subset_df.copy()
customers['age'] = customers['age'].replace(np.nan,round(customer_subset_df['age'].mean()))

# After careful exploration of the data, we have come to the conclusion that
# in the case of FN,Active,club_member_status,fashion_news_frequency 
# nan value is the default value that represents the case of "No Participation"
# For Example, if in Active column, there is a "nan" value then that means that 
# it represents the case where the club member is not active
# So to deal with these default cases, we impute these values with 0 or NONE

customers['club_member_status'] = customers['club_member_status'].replace(np.nan,'NONE')
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace(np.nan,'NONE')
customers['FN'] = customers['FN'].replace(np.nan,0)
customers['Active'] = customers['Active'].replace(np.nan,0)

In [129]:
customers.isna().sum()

customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
postal_code               0
dtype: int64

In [130]:
# Checking for Missing Values in Articles data

article_subset_df.isna().sum()

article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     246
dtype: int64

In [131]:
# Replace Missing descriptions with empty string
articles = article_subset_df.copy()
articles['detail_desc'] = articles['detail_desc'].replace(np.nan,'')

In [132]:
articles.isna().sum()

article_id                      0
product_code                    0
prod_name                       0
product_type_no                 0
product_type_name               0
product_group_name              0
graphical_appearance_no         0
graphical_appearance_name       0
colour_group_code               0
colour_group_name               0
perceived_colour_value_id       0
perceived_colour_value_name     0
perceived_colour_master_id      0
perceived_colour_master_name    0
department_no                   0
department_name                 0
index_code                      0
index_name                      0
index_group_no                  0
index_group_name                0
section_no                      0
section_name                    0
garment_group_no                0
garment_group_name              0
detail_desc                     0
dtype: int64

In [133]:
#Check for missing values in transactions
transactions_train.isna().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [134]:
# Save all the subset data into the csv files 
customers.to_csv("customers_subset.csv",index=False)
articles.to_csv("articles_subset.csv",index=False)
test_data.to_csv("transactions_test_subset.csv",index=False)
transactions_train.to_csv("transactions_train_subset.csv",index=False)